In [2]:

import numpy as np
import pandas as pd



def read_raw(x):

    data = [[] for i in range(6)]
    represent_data_r = []
    represent_data_ir = []
    represent_data_g = []
    ppg_data = []
    time=[]

    imu_freq = 1  # Hz
    imu_start = 14
    imu_data_size = 2  # byte
    imu_type = 3
    imu_period = imu_data_size*imu_type  # xyz
    ppg_freq = 10  # Hz
    ppg_start = imu_start+imu_data_size*3 + (imu_freq-1)*imu_period  # is empty
    ppg_data_size = 4  # byte
    ppg_type = 4
    ppg_period = ppg_data_size*ppg_type  # g/r/ir/e

    rows = x.split('\n')

    for line in rows:
        lines=line
        if line == "":
                continue
        line = bytearray.fromhex(''.join(line.split(' ')[:-1]))
        for i in range(ppg_freq):
            for j in range(ppg_type):
                start = ppg_start+j*ppg_data_size
                end = ppg_start+(j+1)*ppg_data_size
                ppg_data.append(int(int.from_bytes(line[start+i*ppg_period:end+i*ppg_period], byteorder="little")))
            data[0].append(ppg_data[0])
            data[1].append(ppg_data[1])
            data[2].append(ppg_data[2])
            c=lines.split(' ')
            time.append(c[-1:])
    time=[b for a in time for b in a]
    R = np.array(data[0])
    IR = np.array(data[1])
    G = np.array(data[2])
    T=np.array(time)

    df = pd.DataFrame({'R': R,####
                    'IR': IR,
                    'G': G,
                    'T': T})

    df.to_csv('220621_Tai_raw.csv',index=False,header=False)
    
f = open('data/20220621_ike_raw/ppg_raw.txt')
text = f.read()
read_raw(text)


In [6]:
import numpy as np
import pandas as pd
import time
import math
from scipy import signal
import read_ppg


from datetime import datetime
import csv

file = 'data/20220621_Ike/event.csv'
file_apnea = pd.read_csv(file)
file = '220621_Tai_ppgggggg.csv'
file_PPG = pd.read_csv(file, header=None)
file = 'data/20220621_Ike/stage.csv'
file_stage = pd.read_csv(file)

ppgFs = 20

R = np.array(file_PPG.iloc[:, 1])
print(R)
total_sec = np.floor(len(R)/ppgFs).astype(int)
print(total_sec)


st = file_stage['Time [hh:mm:ss.xxx]'][0]
st = datetime.strptime(st[:19], '%Y-%m-%dT%H:%M:%S')
print(st)
start_in_sec = (st.hour*60 + st.minute)*60 + st.second


CSA = 1
OSA = 2
MIX = 2
HYP = 3

apnea = np.zeros(total_sec*2)


[546338 545960 546065 ... 464993 464993 464963]
23947
2022-06-21 22:12:41
